In [1]:
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

Загружаем данные

In [2]:
df = pd.read_csv('All-seasons.csv')

In [3]:
df.head()

,Season,Episode,Character,Line
0,10,1,Stan,"You guys, you guys! Chef is going away. \n"
1,10,1,Kyle,Going away? For how long?\n
2,10,1,Stan,Forever.\n
3,10,1,Chef,I'm sorry boys.\n
4,10,1,Stan,"Chef said he's been bored, so he joining a gro..."


Берем только главных персонажей + делаем стрип строк

In [4]:
main_chars = ['Kyle', 'Cartman', 'Stan', 'Kenny', 'Mr. Garrison']

In [5]:
df = df[df['Character'].isin(main_chars)]

In [6]:
df['Line'] = df['Line'].map(lambda text: text.rstrip())

In [7]:
df.head()

,Season,Episode,Character,Line
0,10,1,Stan,"You guys, you guys! Chef is going away."
1,10,1,Kyle,Going away? For how long?
2,10,1,Stan,Forever.
4,10,1,Stan,"Chef said he's been bored, so he joining a gro..."
9,10,1,Cartman,I'm gonna miss him. I'm gonna miss Chef and I...


In [8]:
df.describe()

,Season,Episode,Character,Line
count,26436,26436,26436,26436
unique,18,18,5,23796
top,2,1,Cartman,What?
freq,2574,2078,9774,203


Делим все на трейн и тест

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df['Line'], df['Character'], test_size=0.2)

Для токенизации используем nltk + не берем знаки препинания

In [10]:
def tokenize(text):
    text = text.lower()
    all_tokens = word_tokenize(text)
    return [token for token in all_tokens if any(char.isalpha() for char in token)]

Со стопсловами лучше, поэтому я их тоже включаю в модель

In [11]:
en_stopwords = stopwords.words('english')

Сначала обучим лес. В выводе выведем общую точность, а также f-меру по каждому классу

In [12]:
clf = Pipeline([
    ('vect', CountVectorizer(
        analyzer=tokenize,
        stop_words=en_stopwords
                            )),
    ('clf', RandomForestClassifier(n_estimators = 200)),
])
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

a = accuracy_score(y_test, y_pred)
f = f1_score(y_test, y_pred, average=None)
print('{:>15}: {}\n'.format('accuracy', a))
print('{:>15}:'.format('f_scores'))
for cl, f_cl in zip(clf.classes_, f):
    print('{:>15}: {}'.format(cl, f_cl))

       accuracy: 0.47655068078668683

       f_scores:
        Cartman: 0.5963963963963964
          Kenny: 0.15023474178403756
           Kyle: 0.380952380952381
   Mr. Garrison: 0.205761316872428
           Stan: 0.42815628027123026


Теперь обучим Байеса

In [13]:
clf = Pipeline([
    ('vect', CountVectorizer(analyzer=tokenize,
                             stop_words=en_stopwords)),
    ('clf', MultinomialNB()),
])
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

a = accuracy_score(y_test, y_pred)
f = f1_score(y_test, y_pred, average=None)
print('{:>15}: {}\n'.format('accuracy', a))
print('{:>15}:'.format('f_scores'))
for cl, f_cl in zip(clf.classes_, f):
    print('{:>15}: {}'.format(cl, f_cl))

       accuracy: 0.48789712556732223

       f_scores:
        Cartman: 0.5995224658128934
          Kenny: 0.13186813186813187
           Kyle: 0.35769561478933787
   Mr. Garrison: 0.1825726141078838
           Stan: 0.46521739130434786


И наконец логит регрессия

In [14]:
clf = Pipeline([
    ('vect', CountVectorizer(analyzer=tokenize,
                             stop_words=en_stopwords)),
    ('clf', LogisticRegression(class_weight='balanced')),
])
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

a = accuracy_score(y_test, y_pred)
f = f1_score(y_test, y_pred, average=None)
print('{:>15}: {}\n'.format('accuracy', a))
print('{:>15}:'.format('f_scores'))
for cl, f_cl in zip(clf.classes_, f):
    print('{:>15}: {}'.format(cl, f_cl))

       accuracy: 0.46501512859304084

       f_scores:
        Cartman: 0.5842880523731587
          Kenny: 0.194647201946472
           Kyle: 0.43445692883895126
   Mr. Garrison: 0.3270440251572327
           Stan: 0.44202652159129546


Из всех трех моделей лучше всего работают лес и Байес. Возможно, они лучше подходят для классификации по множеству классов.
Другое наблюдение: лучше всего модели определяют реплики Картмана, хуже всего -- Кенни. Тут дело конечно еще в том, что у Картмана больше реплик в принципе, но еще может влиять и то, что реплики Кртмана содержат какие-то специфичные ему токены, по которым его удается распознать. 